Import libraries:

In [1]:
import re
import nltk
import pandas as pd
import seaborn as sns
from random import randint

from pathlib import Path

import winsound
import numpy as np
import pandas as pd
from tqdm import tqdm
import networkx as nx
from requests import get
from googlesearch import search

from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
from bs4.element import Comment
from bs4 import BeautifulSoup
import urllib.request

import time
import collections
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from nltk import tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer

from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import chromedriver_binary

Import CDFs:

In [2]:
def tags(x):
    return nltk.pos_tag(nltk.word_tokenize(x))[0][1]

def scoring(a,b):
    try:
        a = a.replace(',', '').split()
        b = b.replace(',', '').split()
        score = int(100*len(set(a).intersection(b))/(len(set(a))))
    
    except:
        score = 0    
        
    return score

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def text_normalize(txt):
    processed_text = re.sub('[^a-zA-Z]', ' ', txt)
    processed_text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",processed_text)
    processed_text=re.sub("(\\d|\\W)+"," ",processed_text)

    stop_words = set(stopwords.words("english"))
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(processed_text)
    
    tokens = [lemmatizer.lemmatize(word) for word in tokens if not word in stop_words]
    tokens = [i for i in tokens if (tags(i) in ['NN', 'NNP', 'NNS', 'NNPS'])]
    
    return tokens

def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

def parse_tds(url):
    htmlString = get(url).text

    soup = BeautifulSoup(htmlString, 'html.parser')
    title = text_from_html(str(soup.find('title')))
    
    title_str = str(title.split('|', 1)[0])
    title_str = title_str.replace('Towards Data Science','')

    keywords = ', '.join([text_from_html(str(i)) for i in soup.find_all('a') if '/tagged' in str(i)]) + ', ' + title_str
    keywords = keywords.lower()
    return keywords

def parse_med(url):
    htmlString = get(url).text
    soup = BeautifulSoup(htmlString, 'html.parser')
    
    title = soup.find('h1')
    title_str = text_from_html(str(title))
    
    keywords = ', '.join([text_from_html(str(i)) for i in soup.find_all('a') if '/tag/' in str(i)]) + ', ' + title_str
    keywords = keywords.lower()
    return keywords

def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

def create_graph(text, nodes=25, max_component=False, common=500):

    ## Normalize text
    tokens = text_normalize(text)
    ## /Normalize text

    ## create graph
    bigrams=list(nltk.ngrams(tokens, 2))
    bigram_counts = collections.Counter(bigrams)

    bigram_df = pd.DataFrame(bigram_counts.most_common(common), columns=['bigram', 'count'])
    d = bigram_df.set_index('bigram').T.to_dict('records')

    G = nx.Graph()
    for k, v in d[0].items():
            G.add_edge(k[0], k[1], weight=(v*10))
    ## /create graph

    ## show max connected component
    if max_component==True:
        G = max(connected_component_subgraphs(G), key=len)
        G = nx.dfs_tree(G)
    ## /show max connected component

    ## remove isolates
    G.remove_nodes_from(list(nx.isolates(G)))
    ## /remove isolates

    ## filter graph by degree
    threshold = int(G.number_of_nodes()*(max(nx.degree_centrality(G).values()))) #max centrality, nodes

    for i in range(0, threshold):
        F = nx.Graph() # empty graph
        fedges = filter(lambda x: G.degree()[x[0]] > 0 and G.degree()[x[1]] > i, G.edges())
        F.add_edges_from(fedges)
        F.remove_nodes_from(list(nx.isolates(F)))

        if F.number_of_nodes() < nodes:
            break
    ## /filter graph by degree

    # pos = nx.spring_layout(F, iterations=500)

    return F

def get_keys(url):
    
    ## parse url
    htmlString = get(url).text
    soup = BeautifulSoup(htmlString, 'html.parser')
    paragraphs = soup.find_all(['p', 'article', 'section', 'title', 'h1', 'h2', 'h3'])
    rawtext = text_from_html(str(paragraphs))
    ## /parse url
    
    ## nlp processing
    tokens = text_normalize(rawtext)
    ## /nlp processing

    ## Bigrams
    keys_list_bg = []
    keys_list_gr = []
    corpus = tokenize.sent_tokenize(' '.join(tokens))
    
    if len(corpus) > 0:
        vec = CountVectorizer(ngram_range=(2,2), max_features=100).fit(corpus)
        bag_of_words = vec.transform(corpus)
        sum_words = bag_of_words.sum(axis=0)
        
        words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
        words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)
        
        keys_list_bg = [str(i[0]) for i in words_freq[:10]]
        keys_list_bg = list(set(keys_list_bg))
        
        F = create_graph(rawtext)
        d = dict(F.degree)
        keys_list_gr = [i[0] for i in sorted(F.degree, key=lambda x: x[1], reverse=True)][:5]
    ## /Bigrams
    
    # mix keys
    keys_list = (keys_list_gr + keys_list_bg)
    keys_cap = (''.join([i.capitalize() for i in ' '.join(keys_list).split()])).split()
    # /mix keys

    keys = ', '.join(keys_list)
    
    return keys, rawtext

### Parse urls:

Create URLs:

In [ ]:
request_list = ['natural language processing', 'text summarization nlp', 'keywords extraction', 'graph analysis nlp']
request_list = request_list + ['text neural network', 'abstractive summarization', 'text lstm', 'text graph analysis']
request_list = request_list + ['text seq to seq', 'text bert', 'gensim summarization', 'tensorflow text summarization']
request_list = request_list + ['website summarizer', 'nlp text representation', 'text clustering']
request_list = request_list + ['nlp chatbot', 'nlp twitter trends', 'deep learning']

Collect Medium and TDS:

In [ ]:
urls = []
SCROLL_PAUSE_TIME = 4

chrome_options = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(chrome_options=chrome_options)
html = driver.find_element_by_tag_name('html')

print('Medium collection:\n')

for i in tqdm(request_list):
    
    driver.get('https://medium.com/search?q=' + i)
    print('Request: '+str(i))
    last_height = driver.execute_script('return document.body.scrollHeight')
    
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        html = driver.find_element_by_tag_name('html')
        html.send_keys(Keys.END)
        html.send_keys(Keys.END)
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:
            break
            
        last_height = new_height    
           
    soup = BeautifulSoup(driver.page_source)
    mydivs = soup.findAll('div', {'class': 'postArticle-content'})
    
    urls = urls + re.findall(r'href=[\'"]?([^\'" >]+)', str(mydivs))
    print('URL number:' + str(len(urls)))
    winsound.Beep(2500, 1000)

print('Towards data science collection:\n')    
    
for i in tqdm(request_list):
    
    driver.get('https://towardsdatascience.com/search?q=' + i)
    print('Request: '+str(i))
    last_height = driver.execute_script('return document.body.scrollHeight')
    
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        html = driver.find_element_by_tag_name('html')
        html.send_keys(Keys.END)
        html.send_keys(Keys.END)
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:
            break
            
        last_height = new_height    
           
    soup = BeautifulSoup(driver.page_source)
    mydivs = soup.findAll('div', {'class': 'postArticle-content'})
    
    urls = urls + re.findall(r'href=[\'"]?([^\'" >]+)', str(mydivs))
    print('URL number:' + str(len(urls)))
    winsound.Beep(2500, 1000)   

Number of unique urls:

In [ ]:
print('Number of inique urls: '+ str(len(set(urls))))

Write urls:

In [ ]:
df = pd.DataFrame(list(set(urls)), columns=['url'])
df.to_excel (r'datasets/urls.xlsx', index = False, header=True)
df.head()

### Get URLs:

In [ ]:
pages_df = pd.read_excel(f'datasets/urls.xlsx', sheet_name='Sheet1')
pages_df.drop_duplicates(subset = 'url', keep = 'first', inplace = True)
pages_list = list(pages_df['url'])

Statistics:

In [ ]:
len(pages_df)

Parse web pages:

In [ ]:
# keys_list = []
# web_list = []

# for i in tqdm(pages_list[:]): 
#     if 'towardsdatascience.com' in str(i):
#             keys_list.append(parse_tds(i))
#             web_list.append(i)
#     elif 'medium.com' in str(i):
#         keys_list.append(parse_med(i))
#         web_list.append(i)
#     else:
#         continue

Completion alert:

In [ ]:
winsound.Beep(2500, 1000) 

Write do dataframe:

In [ ]:
df = pd.DataFrame(list(zip(web_list, keys_list)), columns=['url','tags'])
df.to_excel (r'datasets/keys.xlsx', index = False, header=True)
df.head()

Statistics:

In [ ]:
print('Number of records:', len(df))

### Add ML keys:

Read and filter urls:

In [3]:
df_urls = pd.read_excel(r'datasets/keys.xlsx', sheet_name='Sheet1')
df_urls =  df_urls[df_urls['tags'].str.contains('[A-Za-z]')]
tag_list = list(df_urls['tags'])
pages_list = list(df_urls['url'])
df_urls.head()

,url,tags
0,https://towardsdatascience.com/a-machine-learn...,"machine learning, naturallanguageprocessing, t..."
1,https://towardsdatascience.com/natural-languag...,"machine learning, neural networks, data scienc..."
2,https://medium.com/@swaroopshyam0/combinatory-...,"machine learning, natural language processi, n..."
3,https://medium.com/@sonalimeh1912/text-summary...,"nlp, text summarization, hindi corpus, text su..."
4,https://towardsdatascience.com/building-a-natu...,"machine learning, data, data science, augmenta..."


Statistics:

In [4]:
len(df_urls)

5151

Get ML keys:

In [5]:
keys_list = []
web_list = []
text_list = []

for i in tqdm(pages_list[:]):      
    try:
        keys, text = get_keys(i)     
        keys_list.append(keys)
        text_list.append(text)
        web_list.append(i)   
    except:
        print ('Connection error')
        
    print('Length of list:', len(text_list)) 

  0%|                                                                              | 1/5151 [00:08<12:43:35,  8.90s/it]

Length of list: 1


  0%|                                                                              | 2/5151 [00:17<12:35:36,  8.80s/it]

Length of list: 2


  0%|                                                                              | 3/5151 [00:21<10:38:48,  7.45s/it]

...

Length of list: 5148


100%|███████████████████████████████████████████████████████████████████████████▉| 5150/5151 [8:42:32<00:05,  5.62s/it]

Length of list: 5149


100%|████████████████████████████████████████████████████████████████████████████| 5151/5151 [8:42:35<00:00,  6.09s/it]

Length of list: 5150


Completion alert:

In [6]:
winsound.Beep(2500, 1000) 

Write tags and keys to df:

In [7]:
df = pd.DataFrame(list(zip(web_list, tag_list, text_list, keys_list)), columns=['url','tags','text','keys'])
df.to_excel (r'datasets/keys_full.xlsx', index = False, header=True)
df.head()

C:\Users\skamenshchikov\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:937: UserWarning: Ignoring URL 'https://pixabay.com%20How%20Recurrent%20Neural%20Networks%20work%20Simeon%20Kostadinov%20Follow%20Dec%202,%202017%20%20·%20%206%20min%20read%20You%20have%20definitely%20come%20across%20software%20that%20translates%20natural%20language%20(Google%20Translate)%20or%20turns%20your%20speech%20into%20text%20(Apple%20Siri)%20and%20probably,%20at%20first,%20you%20were%20curious%20how%20it%20works.%20In%20the%20last%20couple%20of%20years,%20a%20considerable%20improvement%20in%20the%20science%20behind%20these%20systems%20has%20taken%20place.%20For%20example,%20in%20late%202016,%20Google%20introduced%20a%20new%20system%20behind%20their%20Google%20Translate%20which%20uses%20state-of-the-art%20machine%20learning%20techniques.%20The%20improvement%20is%20remarkable%20and%20you%20can%20test%20it%20yourself.%20Another%20astonishing%20example%20is%20Baidu’s%20most%20recent%20text%20to%20speech%20:

C:\Users\skamenshchikov\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:937: UserWarning: Ignoring URL 'http://cf.ydcdn.net/latest/images/wordfinder-background.jpg%20Glove%20Research%20Paper%20Clearly%20Explained%20Intuitive%20understanding%20and%20mathematical%20derivations%20used%20in%20Glove.%20Meesala%20Lokesh%20Follow%20Jul%205%20%20·%20%209%20min%20read%20It%20is%20important%20for%20a%20data%20scientist%20to%20read%20and%20understand%20research%20papers%20that%20form%20the%20base%20of%20the%20current%20start%20of%20the%20art%20models%20be%20it%20in%20Natural%20Language%20Processing%20or%20Computer%20Vision%20which%20helps%20us%20solve%20many%20tasks.%20It%20helps%20us%20to%20think%20and%20understand%20what%20goes%20into%20the%20research%20that%20gave%20us%20the%20models%20and%20algorithms%20we%20use%20in%20our%20data%20science%20journey.%20Often%20research%20papers%20come%20across%20as%20math-intensive%20scripts%20so%20we%20tend%20to%20ignore%20them%20altogether.%20In%20my%20

,url,tags,text,keys
0,https://towardsdatascience.com/a-machine-learn...,"machine learning, naturallanguageprocessing, t...",A Machine Learning Approach to Author Identifi...,"word, author, novel, text, Machine, text snipp..."
1,https://towardsdatascience.com/natural-languag...,"machine learning, neural networks, data scienc...",Natural Language Processing: From Basics to us...,"word, language, Language, sentence, length, cr..."
2,https://medium.com/@swaroopshyam0/combinatory-...,"machine learning, natural language processi, n...",Combinatory Categorical Grammar (CCG) for NLU ...,"utterance, e, CCG, state, phrase, np lambda, p..."
3,https://medium.com/@sonalimeh1912/text-summary...,"nlp, text summarization, hindi corpus, text su...",Text summary for Hindi Corpus using NLP Sonali...,"sentence, text, document, word, corpus, senten..."
4,https://towardsdatascience.com/building-a-natu...,"machine learning, data, data science, augmenta...",Building a Natural Language Lesson Jake Young ...,"method, model, synonym, machine, amount, train..."


#### Similarity analysis

Define score:

In [17]:
df = pd.read_excel(r'datasets/keys_full.xlsx', sheet_name='Sheet1')
df['keys'] = df['keys'].str.lower()
df['tags'] = df['tags'].str.lower()

df['recall'] = df.apply(lambda x: scoring(x['tags'], x['keys']), axis=1)
df['precision'] = df.apply(lambda x: scoring(x['keys'], x['tags']), axis=1)
df['F1'] = 2*(df['recall']*df['precision'])/(df['recall']+df['precision'])

df.dropna(inplace=True)
df['F1'] = df['F1'].astype(int)

df.head()

,url,tags,text,keys,recall,precision,F1
0,https://towardsdatascience.com/a-machine-learn...,"machine learning, naturallanguageprocessing, t...",A Machine Learning Approach to Author Identifi...,"word, author, novel, text, machine, text snipp...",25,9,13
1,https://towardsdatascience.com/natural-languag...,"machine learning, neural networks, data scienc...",Natural Language Processing: From Basics to us...,"word, language, language, sentence, length, cr...",12,5,7
2,https://medium.com/@swaroopshyam0/combinatory-...,"machine learning, natural language processi, n...",Combinatory Categorical Grammar (CCG) for NLU ...,"utterance, e, ccg, state, phrase, np lambda, p...",13,14,13
3,https://medium.com/@sonalimeh1912/text-summary...,"nlp, text summarization, hindi corpus, text su...",Text summary for Hindi Corpus using NLP Sonali...,"sentence, text, document, word, corpus, senten...",62,41,49
4,https://towardsdatascience.com/building-a-natu...,"machine learning, data, data science, augmenta...",Building a Natural Language Lesson Jake Young ...,"method, model, synonym, machine, amount, train...",33,11,16


Statistics:

In [18]:
len(df)

3983

Write new dataframe:

In [19]:
df.to_excel (r'datasets/scoring.xlsx', index = False, header=True)
df.head()

,url,tags,text,keys,recall,precision,F1
0,https://towardsdatascience.com/a-machine-learn...,"machine learning, naturallanguageprocessing, t...",A Machine Learning Approach to Author Identifi...,"word, author, novel, text, machine, text snipp...",25,9,13
1,https://towardsdatascience.com/natural-languag...,"machine learning, neural networks, data scienc...",Natural Language Processing: From Basics to us...,"word, language, language, sentence, length, cr...",12,5,7
2,https://medium.com/@swaroopshyam0/combinatory-...,"machine learning, natural language processi, n...",Combinatory Categorical Grammar (CCG) for NLU ...,"utterance, e, ccg, state, phrase, np lambda, p...",13,14,13
3,https://medium.com/@sonalimeh1912/text-summary...,"nlp, text summarization, hindi corpus, text su...",Text summary for Hindi Corpus using NLP Sonali...,"sentence, text, document, word, corpus, senten...",62,41,49
4,https://towardsdatascience.com/building-a-natu...,"machine learning, data, data science, augmenta...",Building a Natural Language Lesson Jake Young ...,"method, model, synonym, machine, amount, train...",33,11,16


Statistics:

In [20]:
print('Median score:', int(np.median(df['F1'])), '%')
print('Standard deviation:', int(np.std(df['F1'])), '%')
print('Max/min scattering:', int((np.max(df['F1'])-np.min(df['F1']))/2), '%')
print('Number of records:', len(df))

Median score: 15 %
Standard deviation: 9 %
Max/min scattering: 31 %
Number of records: 3983
